In [5]:
from google.cloud import bigquery
import pandas as pd 
import chime   
import seaborn as sns
client = bigquery.Client("gcp-wow-rwds-ai-checkout-dev")

/opt/homebrew/lib/python3.11/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [6]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

In [ ]:
check = client.query(
"""
SELECT 
    campaign_id, 
    COUNT(DISTINCT test_store) AS n_test_stores, 
    COUNT(DISTINCT control_store) AS n_control_stores 
FROM gcp-wow-cart-data-dev-d4d7.davide.digital_screens_store_comparisons_plus_baseline_4 
GROUP BY 1 
"""
).result()
check_df = check.to_dataframe()
check_df 

In [7]:
check = client.query(
"""

CREATE OR REPLACE TABLE gcp-wow-cart-data-dev-d4d7.davide.digital_screens_store_similarity AS
    WITH 

    n_days AS (
        SELECT 
            campaign_id, 
            MAX(n_days_campaign_period) AS max_days_campaign_period 
        FROM  gcp-wow-cart-data-dev-d4d7.davide.instore_screens_sales_pre_vs_during_period_plus_baseline_4
        GROUP BY 1
    ),

    step_two AS (
        SELECT 
            res.*, 
            SAFE_DIVIDE(total_sales_campaign_period , total_sales_pre_period) - 1 AS perc_sales_uplift,
            total_sales_campaign_period - total_sales_pre_period AS raw_sales_uplift
        FROM gcp-wow-cart-data-dev-d4d7.davide.instore_screens_sales_pre_vs_during_period_plus_baseline_4 res
        LEFT JOIN n_days
            ON res.campaign_id = n_days.campaign_id 
            AND res.n_days_campaign_period = n_days.max_days_campaign_period 
        WHERE max_days_campaign_period IS NOT NULL
        AND total_baskets_campaign_period / (n_days_campaign_period / 7) >= 100 
        AND total_baskets_pre_period / (n_days_campaign_period / 7) >= 100 
    ),

    step_three AS (
        SELECT
            test.campaign_id, 
            test.media_start_date,
            test.media_end_date,
            test.Site AS test_store, 
            control.Site AS control_store,
            test.perc_sales_uplift AS test_store_perc_uplift, 
            control.perc_sales_uplift AS control_store_perc_uplift, 
            test.raw_sales_uplift AS test_store_raw_uplift,
            control.raw_sales_uplift AS control_store_raw_uplift,
            test.total_sales_pre_period AS test_store_pre_sales,
            control.total_sales_pre_period AS control_store_pre_sales,
            test.total_sales_campaign_period AS test_store_campaign_sales, 
            control.total_sales_campaign_period AS control_store_campaign_sales
        FROM step_two test
        LEFT JOIN step_two control 
            ON test.campaign_id = control.campaign_id 
            AND test.Site <> control.Site 
        WHERE test.test_or_control = "Test" 
        AND control.test_or_control = "Control" 
    )
    , 

    step_four AS (
        SELECT 
            step_three.*,
            SAFE_DIVIDE(baseline_test.sales_amount , 12) AS test_mean_historical_sales,
            SAFE_DIVIDE(baseline_control.sales_amount , 12) AS control_mean_historical_Sales, 
            SAFE_DIVIDE(SAFE_DIVIDE(baseline_test.sales_amount , 12) , SAFE_DIVIDE(baseline_control.sales_amount , 12)) -1 AS test_vs_control_mean_historical_sales_perc_diff, 
            baseline_test.stddev_sales_amount AS test_stddev_sales_amount, 
            baseline_control.stddev_sales_amount AS control_stddev_sales_amount,
            SAFE_DIVIDE(baseline_test.stddev_sales_amount , baseline_control.stddev_sales_amount) -1 AS test_vs_control_stddev_sales_amount_perc_diff, 
            ABS(SAFE_DIVIDE(SAFE_DIVIDE(baseline_test.sales_amount , 12) , SAFE_DIVIDE(baseline_control.sales_amount , 12)) -1) + ABS(SAFE_DIVIDE(baseline_test.stddev_sales_amount , baseline_control.stddev_sales_amount) -1) AS sum_of_abs_perc_diffs
        FROM step_three 
        
        LEFT JOIN gcp-wow-cart-data-dev-d4d7.davide.baseline_statistics_with_campaign_3 baseline_test 
            ON step_three.test_store = baseline_test.Site AND step_three.campaign_id = baseline_test.campaign_id

        LEFT JOIN gcp-wow-cart-data-dev-d4d7.davide.baseline_statistics_with_campaign_3 baseline_control 
            ON step_three.control_store = baseline_control.Site AND step_three.campaign_id = baseline_control.campaign_id
 
    ),
    step_five AS (

        SELECT
            campaign_id, 
            test_store, 
            control_store,
            test_mean_historical_sales,
            control_mean_historical_Sales,
            test_stddev_sales_amount,
            control_stddev_sales_amount,
            test_vs_control_mean_historical_sales_perc_diff,
            test_vs_control_stddev_sales_amount_perc_diff,
            sum_of_abs_perc_diffs,
            ROW_NUMBER() OVER(PARTITION BY campaign_id, test_store ORDER BY sum_of_abs_perc_diffs) AS row_num
        FROM step_four
        WHERE sum_of_abs_perc_diffs IS NOT NULL
        ORDER BY 1,2,sum_of_abs_perc_diffs
    ),
    step_six AS (
        SELECT
            step_five.*,
            ROW_NUMBER() OVER(PARTITION BY campaign_id, control_store ORDER BY sum_of_abs_perc_diffs) AS control_store_occurrence
        FROM step_five
        WHERE row_num = 1
    ),

    store_counts AS (
    
        SELECT 
            llama.campaign_id,
            COUNT(DISTINCT test_store) AS test_stores_matched,
            test_stores_unmatched,
            COUNT(DISTINCT control_store) AS control_stores_matched,
            control_stores_unmatched
        FROM step_six
        LEFT JOIN (
            SELECT 
                campaign_id, 
                COUNT(DISTINCT test_store) AS test_stores_unmatched,
                COUNT(DISTINCT control_store) AS control_stores_unmatched
            FROM step_three
            GROUP BY 1
        ) unmatched ON step_six.campaign_id = unmatched.campaign_id
        WHERE control_store_occurrence = 1
        GROUP BY 1,3,5
    )

    SELECT 
        step_four.*,

    FROM step_four 
    INNER JOIN step_six 
        ON step_six.campaign_id = step_four.campaign_id 
        AND step_six.control_store = step_four.control_store
        AND step_six.test_store = step_four.test_store
    WHERE
        step_four.campaign_id IN (SELECT DISTINCT campaign_id FROM store_counts WHERE test_stores_matched >= 50)
    AND 
        step_six.control_store_occurrence = 1
    ORDER BY
        campaign_id, test_store, sum_of_abs_perc_diffs
;

"""
).result()
check_df = check.to_dataframe()
check_df 

""


In [10]:
check = client.query(
"""
CREATE OR REPLACE TABLE gcp-wow-cart-data-dev-d4d7.davide.digital_screens_store_similarity_underpowered AS
    WITH 

    n_days AS (
        SELECT 
            campaign_id, 
            MAX(n_days_campaign_period) AS max_days_campaign_period 
        FROM  gcp-wow-cart-data-dev-d4d7.davide.instore_screens_sales_pre_vs_during_period_plus_baseline_4
        GROUP BY 1
    ),

    step_two AS (
        SELECT 
            res.*, 
            SAFE_DIVIDE(total_sales_campaign_period , total_sales_pre_period) - 1 AS perc_sales_uplift,
            total_sales_campaign_period - total_sales_pre_period AS raw_sales_uplift
        FROM gcp-wow-cart-data-dev-d4d7.davide.instore_screens_sales_pre_vs_during_period_plus_baseline_4 res
        LEFT JOIN n_days
            ON res.campaign_id = n_days.campaign_id 
            AND res.n_days_campaign_period = n_days.max_days_campaign_period 
        WHERE max_days_campaign_period IS NOT NULL
        --AND total_baskets_campaign_period / (n_days_campaign_period / 7) >= 100 
        --AND total_baskets_pre_period / (n_days_campaign_period / 7) >= 100 
    ),

    step_three AS (
        SELECT
            test.campaign_id, 
            test.media_start_date,
            test.media_end_date,
            test.Site AS test_store, 
            control.Site AS control_store,
            test.perc_sales_uplift AS test_store_perc_uplift, 
            control.perc_sales_uplift AS control_store_perc_uplift, 
            test.raw_sales_uplift AS test_store_raw_uplift,
            control.raw_sales_uplift AS control_store_raw_uplift,
            test.total_sales_pre_period AS test_store_pre_sales,
            control.total_sales_pre_period AS control_store_pre_sales,
            test.total_sales_campaign_period AS test_store_campaign_sales, 
            control.total_sales_campaign_period AS control_store_campaign_sales
        FROM step_two test
        LEFT JOIN step_two control 
            ON test.campaign_id = control.campaign_id 
            AND test.Site <> control.Site 
        WHERE test.test_or_control = "Test" 
        AND control.test_or_control = "Control" 
    )
    , 

    step_four AS (
        SELECT 
            step_three.*,
            SAFE_DIVIDE(baseline_test.sales_amount , 12) AS test_mean_historical_sales,
            SAFE_DIVIDE(baseline_control.sales_amount , 12) AS control_mean_historical_Sales, 
            SAFE_DIVIDE(SAFE_DIVIDE(baseline_test.sales_amount , 12) , SAFE_DIVIDE(baseline_control.sales_amount , 12)) -1 AS test_vs_control_mean_historical_sales_perc_diff, 
            baseline_test.stddev_sales_amount AS test_stddev_sales_amount, 
            baseline_control.stddev_sales_amount AS control_stddev_sales_amount,
            SAFE_DIVIDE(baseline_test.stddev_sales_amount , baseline_control.stddev_sales_amount) -1 AS test_vs_control_stddev_sales_amount_perc_diff, 
            ABS(SAFE_DIVIDE(SAFE_DIVIDE(baseline_test.sales_amount , 12) , SAFE_DIVIDE(baseline_control.sales_amount , 12)) -1) + ABS(SAFE_DIVIDE(baseline_test.stddev_sales_amount , baseline_control.stddev_sales_amount) -1) AS sum_of_abs_perc_diffs
        FROM step_three 
        
        LEFT JOIN gcp-wow-cart-data-dev-d4d7.davide.baseline_statistics_with_campaign_3 baseline_test 
            ON step_three.test_store = baseline_test.Site AND step_three.campaign_id = baseline_test.campaign_id

        LEFT JOIN gcp-wow-cart-data-dev-d4d7.davide.baseline_statistics_with_campaign_3 baseline_control 
            ON step_three.control_store = baseline_control.Site AND step_three.campaign_id = baseline_control.campaign_id
 
    ),
    giraffe AS (

        SELECT
            campaign_id, 
            test_store, 
            control_store,
            test_mean_historical_sales,
            control_mean_historical_Sales,
            test_stddev_sales_amount,
            control_stddev_sales_amount,
            test_vs_control_mean_historical_sales_perc_diff,
            test_vs_control_stddev_sales_amount_perc_diff,
            sum_of_abs_perc_diffs,
            ROW_NUMBER() OVER(PARTITION BY campaign_id, test_store ORDER BY sum_of_abs_perc_diffs) AS row_num
        FROM step_four
        WHERE sum_of_abs_perc_diffs IS NOT NULL
        ORDER BY 1,2,sum_of_abs_perc_diffs
    ),
    llama AS (
        SELECT
            giraffe.*,
            ROW_NUMBER() OVER(PARTITION BY campaign_id, control_store ORDER BY sum_of_abs_perc_diffs) AS control_store_occurrence
            --unmatched.campaign_id, 
            --COUNT(DISTINCT test_store) AS test_stores_matched,
            --test_stores_unmatched,
            --COUNT(DISTINCT control_store) AS control_stores_matched,
            --control_stores_unmatched
        FROM giraffe
        WHERE row_num = 1
    ),

    store_counts AS (
    
        SELECT 
            llama.campaign_id,
            COUNT(DISTINCT test_store) AS test_stores_matched,
            test_stores_unmatched,
            COUNT(DISTINCT control_store) AS control_stores_matched,
            control_stores_unmatched
        FROM llama
        LEFT JOIN (
            SELECT 
                campaign_id, 
                COUNT(DISTINCT test_store) AS test_stores_unmatched,
                COUNT(DISTINCT control_store) AS control_stores_unmatched
            FROM step_three
            GROUP BY 1
        ) unmatched ON llama.campaign_id = unmatched.campaign_id
        WHERE control_store_occurrence = 1
        GROUP BY 1,3,5
    )

    SELECT 
        step_four.*,

    FROM step_four 
    INNER JOIN llama 
        ON llama.campaign_id = step_four.campaign_id 
        AND llama.control_store = step_four.control_store
        AND llama.test_store = step_four.test_store
    WHERE
        step_four.campaign_id IN (SELECT DISTINCT campaign_id FROM store_counts WHERE test_stores_matched >= 50)
    AND 
        llama.control_store_occurrence = 1
    ORDER BY
        campaign_id, test_store, sum_of_abs_perc_diffs
;

"""
).result()
check_df = check.to_dataframe()
check_df 

""


In [11]:
check = client.query(
"""
with step_one AS (
SELECT 
    a.campaign_id, 
    m_type.media_type,
    a.media_start_date,
    a.media_end_date,
    test_store,
    control_store, 
    sum_of_abs_perc_diffs,
    test_vs_control_mean_historical_sales_perc_diff,
    test_vs_control_stddev_sales_amount_perc_diff,
    test_store_pre_sales,
    control_store_pre_sales,
    test_store_campaign_sales,
    control_store_campaign_sales,
    test_store_raw_uplift,
    control_store_raw_uplift,
    test_mean_historical_sales,
    control_mean_historical_Sales,
    test_stddev_sales_amount AS test_stddev_sales,
    control_stddev_sales_amount AS control_stddev_sales, 
    ((test_store_campaign_sales - test_mean_historical_sales) / test_stddev_sales_amount) * test_mean_historical_sales AS test_normalized_uplift,
    ((control_store_campaign_sales - control_mean_historical_sales) / control_stddev_sales_amount) * control_mean_historical_sales AS control_normalized_uplift,
    (((test_store_campaign_sales - test_mean_historical_sales) / test_stddev_sales_amount) * test_mean_historical_sales) - (((control_store_campaign_sales - control_mean_historical_sales) / control_stddev_sales_amount) * control_mean_historical_sales) AS normalized_uplift_raw_diff,
FROM gcp-wow-cart-data-dev-d4d7.davide.digital_screens_store_similarity a
LEFT JOIN gcp-wow-cart-data-dev-d4d7.davide.instore_screens_campaigns_june_2023_onwards_2 m_type 
    ON a.campaign_id = m_type.booking_and_asset_number
WHERE a.campaign_id = "WOW20000332_1"
),
step_two AS (
SELECT step_one.*,
    CASE WHEN test_normalized_uplift < 0 OR control_normalized_uplift < 0 THEN (SAFE_DIVIDE(test_normalized_uplift , control_normalized_uplift) - 1) * -1 ELSE (SAFE_DIVIDE( test_normalized_uplift, control_normalized_uplift) -1) END AS percentage_diff_in_normalized_uplift
FROM step_one
)
SELECT 
    AVG(percentage_diff_in_normalized_uplift) AS avg_perc_diff
FROM step_two
"""
).result()
check_df = check.to_dataframe()
check_df 

,avg_perc_diff
0,0.290872


In [9]:
check = client.query(
"""
with step_one AS (
SELECT 
    a.campaign_id, 
    m_type.media_type,
    a.media_start_date,
    a.media_end_date,
    test_store,
    control_store, 
    sum_of_abs_perc_diffs,
    test_vs_control_mean_historical_sales_perc_diff,
    test_vs_control_stddev_sales_amount_perc_diff,
    test_store_pre_sales,
    control_store_pre_sales,
    test_store_campaign_sales,
    control_store_campaign_sales,
    test_store_raw_uplift,
    control_store_raw_uplift,
    test_mean_historical_sales,
    control_mean_historical_Sales,
    test_stddev_sales_amount AS test_stddev_sales,
    control_stddev_sales_amount AS control_stddev_sales, 
    ((test_store_campaign_sales - test_mean_historical_sales) / test_stddev_sales_amount) * test_mean_historical_sales AS test_normalized_uplift,
    ((control_store_campaign_sales - control_mean_historical_sales) / control_stddev_sales_amount) * control_mean_historical_sales AS control_normalized_uplift,
    (((test_store_campaign_sales - test_mean_historical_sales) / test_stddev_sales_amount) * test_mean_historical_sales) - (((control_store_campaign_sales - control_mean_historical_sales) / control_stddev_sales_amount) * control_mean_historical_sales) AS normalized_uplift_raw_diff,
FROM gcp-wow-cart-data-dev-d4d7.davide.digital_screens_store_similarity a
LEFT JOIN gcp-wow-cart-data-dev-d4d7.davide.instore_screens_campaigns_june_2023_onwards_2 m_type 
    ON a.campaign_id = m_type.booking_and_asset_number
)
--step_two AS (
SELECT step_one.*,
    CASE WHEN test_store_raw_uplift < 0 OR control_store_raw_uplift < 0 THEN (SAFE_DIVIDE(test_store_raw_uplift , control_store_raw_uplift) - 1) * -1 ELSE (SAFE_DIVIDE( test_store_raw_uplift, control_store_raw_uplift) -1) END AS percentage_diff_in_raw_uplift,
    CASE WHEN test_normalized_uplift < 0 OR control_normalized_uplift < 0 THEN (SAFE_DIVIDE(test_normalized_uplift , control_normalized_uplift) - 1) * -1 ELSE (SAFE_DIVIDE( test_normalized_uplift, control_normalized_uplift) -1) END AS percentage_diff_in_normalized_uplift
FROM step_one
WHERE campaign_id = "WOW20007647_3"
--)
/*
--effects AS (
SELECT 
    campaign_id,
    COUNT(DISTINCT test_store) AS total_stores,
    AVG(percentage_diff_in_raw_uplift) AS avg_perc_diff_raw_uplift, 
    AVG(percentage_diff_in_normalized_uplift) AS avg_perc_diff_normalized_uplift
FROM step_two
GROUP BY 1

), 
stats AS (
    SELECT 
        AVG(avg_perc_diff_raw_uplift) AS mean_effect_size, 
        STDDEV(avg_perc_diff_raw_uplift) AS stddev_effect_size
    FROM effects
),
step_three AS (
SELECT 
    effects.* 
FROM effects
CROSS JOIN stats 
WHERE ABS(effects.avg_perc_diff_raw_uplift) < (mean_effect_size + (2 * stddev_effect_size))
AND ABS(effects.avg_perc_diff_normalized_uplift) < (mean_effect_size + (2 * stddev_effect_size))
) 
SELECT 
    AVG(avg_perc_diff_normalized_uplift) AS normalized_effect,
    AVG(avg_perc_diff_raw_uplift) AS raw_effect
FROM step_three
*/
"""
).result()
check_df = check.to_dataframe()
check_df.to_csv("~/Downloads/Screens Analysis Raw Slice for Munz and Saurabh.csv")

In [16]:
check = client.query(
"""

SELECT 
    campaign_id,
    test_store,
    control_store, 
    test_vs_control_mean_historical_sales_perc_diff,
    ROW_NUMBER() OVER(PARTITION BY control_store ORDER BY test_vs_control_mean_historical_sales_perc_diff) AS abs_perc_diff_rank 
FROM gcp-wow-cart-data-dev-d4d7.davide.digital_screens_store_comparisons_plus_baseline_4 
WHERE test_mean_historical_sales IS NOT NULL 
AND control_mean_historical_Sales IS NOT NULL 
AND test_stddev_sales_amount IS NOT NULL 
AND control_stddev_sales_amount IS NOT NULL
ORDER BY campaign_id, control_store, abs_perc_diff_rank
LIMIT 400
"""
).result()
check_df = check.to_dataframe()
check_df 

,campaign_id,test_store,control_store,test_vs_control_mean_historical_sales_perc_diff,abs_perc_diff_rank
0,WOW20000332_1,4155,1647,0.081623366,1
1,WOW20000332_1,1625,1647,0.084814959,2
2,WOW20000332_1,3349,1647,0.085345054,3
3,WOW20000332_1,2867,1647,0.088416346,4
4,WOW20000332_1,3189,1647,0.088772938,5
5,WOW20000332_1,2706,1647,0.089623438,6
6,WOW20000332_1,2076,1647,0.097024159,7
7,WOW20000332_1,3147,1647,0.097901739,8
8,WOW20000332_1,1204,1647,0.099448648,9
9,WOW20000332_1,3091,1647,0.105386575,10


In [10]:
check = client.query(
"""
with step_one AS (
SELECT 
    *,
    ROW_NUMBER() OVER(PARTITION BY campaign_id, control_store ORDER BY test_vs_control_mean_historical_sales_perc_diff) AS abs_perc_diff_rank 
FROM gcp-wow-cart-data-dev-d4d7.davide.digital_screens_store_comparisons_plus_baseline_4 
WHERE test_mean_historical_sales IS NOT NULL 
AND control_mean_historical_Sales IS NOT NULL 
AND test_stddev_sales_amount IS NOT NULL 
AND control_stddev_sales_amount IS NOT NULL
),
step_two AS (
    SELECT * FROM step_one WHERE abs_perc_diff_rank = 1
)
--step_three AS (
    SELECT 
        step_one.* 
    FROM step_one 
    LEFT JOIN step_two 
        ON step_one.campaign_id = step_two.campaign_id 
        AND step_one.test_store = step_two.test_store 
        AND step_one.control_store = step_two.control_store 
    WHERE step_two.control_store IS NOT NULL
    ORDER BY step_one.campaign_id, step_one.control_store
/*) 
SELECT 
    campaign_id, 
    COUNT(DISTINCT test_store) AS n_test_stores, 
    COUNT(DISTINCT control_store) AS n_control_stores
FROM step_three
GROUP BY 1
 */
 LIMIT 20
"""
).result()
check_df = check.to_dataframe()
check_df 

,campaign_id,media_start_date,media_end_date,test_store,control_store,test_store_perc_uplift,control_store_perc_uplift,test_store_raw_uplift,control_store_raw_uplift,test_store_pre_sales,control_store_pre_sales,test_store_campaign_sales,control_store_campaign_sales,test_mean_historical_sales,control_mean_historical_Sales,test_vs_control_mean_historical_sales_perc_diff,test_stddev_sales_amount,control_stddev_sales_amount,test_vs_control_stddev_sales_amount_perc_diff,sum_of_abs_perc_diffs,sim_rank,test_store_mean_transactions,test_store_stddev_transactions,test_store_stddev_sales_amount,test_store_variance_transactions,control_store_mean_transactions,control_store_stddev_transactions,control_store_stddev_sales_amount,control_store_variance_transactions,perc_uplift_effect,raw_uplift_effect,significance,test_store_sales_amount,control_store_sales_amount,abs_perc_diff_rank
0,WOW20000332_1,2023-05-03,2023-05-09,4155,1647,1.091068386,1.115631997,224.160000000,2740.550000000,205.450000000,2456.500000000,429.610000000,5197.050000000,283.834166667,3477.364166667,0.081623366,109.273732,989.197170,0.110467,0.192090,1,43.384615,18.136237,109.273732,328.923077,434.384615,138.077598,989.197170,19065.423077,-0.024563611,-2516.390000000,Significant,3406.010000000,41728.370000000,1
1,WOW20000338_1,2023-05-10,2023-05-16,2941,1199,0.221010931,0.101040694,74.200000000,430.690000000,335.730000000,4262.540000000,409.930000000,4693.230000000,264.338333333,4042.721666667,0.065386231,61.294497,828.826515,0.073953,0.139340,1,19.384615,5.679473,61.294497,32.256410,237.461538,52.897094,828.826515,2798.102564,0.119970237,-356.490000000,Significant,3172.060000000,48512.660000000,1
2,WOW20000338_2,2023-05-10,2023-05-16,3368,1318,0.223309428,0.204625972,76.910000000,642.280000000,344.410000000,3138.800000000,421.320000000,3781.080000000,293.030000000,4101.155833333,0.071450589,70.253034,975.293564,0.072033,0.143483,1,22.000000,6.137318,70.253034,37.666667,261.461538,62.051881,975.293564,3850.435897,0.018683456,-565.370000000,Significant,3516.360000000,49213.870000000,1
3,WOW20000382_1,2023-04-19,2023-04-25,3779,1092,1.580712738,0.444042336,656.470000000,1154.590000000,415.300000000,2600.180000000,1071.770000000,3754.770000000,413.816666667,2960.410833333,0.139783527,87.886017,745.730707,0.117852,0.257636,1,78.538462,22.681179,87.886017,514.435897,350.538462,94.367734,745.730707,8905.269231,1.136670402,-498.120000000,Significant,4965.800000000,35524.930000000,1
4,WOW20000382_2,2023-04-26,2023-05-02,3138,1099,-0.368619857,-0.401711674,-270.320000000,-1556.460000000,733.330000000,3874.570000000,463.010000000,2318.110000000,384.717500000,3041.345833333,0.126495808,88.862958,611.733757,0.145264,0.271760,1,64.461538,12.587132,88.862958,158.435897,382.538462,74.018709,611.733757,5478.769231,0.033091817,1286.140000000,Significant,4616.610000000,36496.150000000,1
5,WOW20000466_1,2023-05-10,2023-05-16,2894,1649,1.288499025,0.198489613,46.270000000,77.010000000,35.910000000,387.980000000,82.180000000,464.990000000,53.415833333,326.431666667,0.163635575,30.513491,88.854207,0.343411,0.507046,1,7.307692,3.682948,30.513491,13.564103,43.769231,12.852716,88.854207,165.192308,1.090009412,-30.740000000,Significant,640.990000000,3917.180000000,1
6,WOW20000487_1,2023-04-12,2023-04-18,7012,1147,4.441558442,3.978014373,164.160000000,747.270000000,36.960000000,187.850000000,201.120000000,935.120000000,103.557500000,663.932500000,0.155975946,47.068426,482.329466,0.097586,0.253562,1,13.846154,8.274924,47.068426,68.474359,81.769231,77.674700,482.329466,6033.358974,0.463544069,-583.110000000,Significant,1242.690000000,7967.190000000,1
7,WOW20000487_1,2023-04-12,2023-04-18,2559,1759,2.572390020,2.681047598,470.130000000,740.130000000,182.760000000,276.060000000,652.890000000,1016.190000000,319.853333333,774.780833333,0.412830725,102.646118,371.810625,0.276071,0.688902,1,36.461538,18.200803,102.646118,331.269231,93.692308,61.177862,371.810625,3742.730769,-0.108657578,-270.000000000,Signif

In [ ]:
check = client.query(
"""
"""
).result()
check_df = check.to_dataframe()
check_df 